# DSPy-Clarifai lm and retriever example notebook

This notebook will walk you through on the integration of clarifai into DSPy which enables the DSPy users to leverage clarifai capabilities of calling llm models from clarifai platform and to utilize clarifai app as retriever for their vector search use cases.

### Setup

In [ ]:
!pip install clarifai

Import necessary packages

In [7]:
import dspy
from dspy.retrieve.clarifai_rm import ClarifaiRM 

Initialize clarifai app id, user id and PAT.

You can browse the portal to obtain [MODEL URL](https://clarifai.com/explore/models) for different models in clarifai community.

In [8]:
#for the demo we are going with llama2-70b-chat
MODEL_URL = "https://clarifai.com/meta/Llama-2/models/llama2-70b-chat" 
PAT = CLARIFAI_PAT
USER_ID = "YOUR_USER_ID"
APP_ID = "YOUR_APP_ID"

Initialize LLM class

Make sure to pass all the model parameters in inference_params field of clarifaiLLM class. 

In [10]:

llm=dspy.Clarifai(model=MODEL_URL, api_key=PAT, n=2, inference_params={"max_tokens":100,'temperature':0.6})

Initialize Clarifai Retriever model class


In [12]:

retriever_model=ClarifaiRM(clarifai_user_id=USER_ID, clarfiai_app_id=APP_ID, clarifai_pat=PAT)

configure dspy with llm and rm models.

In [13]:
dspy.settings.configure(lm=llm, rm=retriever_model)

### Example: dspy.signature and dspy.module with clairfaiLLM

In [18]:
sentence = "disney again ransacks its archives for a quick-buck sequel ."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment')
print(classify(sentence=sentence).sentiment)

NEGATIVE




### Example: Quick glimpse into how our retriever works when a query is passed to the dspy.Retrieve class

In [21]:
retrieve = dspy.Retrieve(k=1)
topK_passages = retrieve("what are the stages in planning, sanctioning and execution of public works").passages

In [23]:
print(topK_passages)

['1.11\n\nProcessing of Public Works\n\nFollowing are the stages in planning, sanctioning and execution of work.\ni)\n\nPerspective Planning for works;\n\nii)\n\nPreparation of Preliminary Project Report (PPR) or Rough Cost Estimate;\n\niii) Acceptance of necessity and issue of in-Principle Approval;\niv) Preparation of Detailed Project Report (DPR) or Preliminary Estimate (PE);\n\n15']


## RAG dspy module using clarifai as retriever

Generally to construct a module in dspy, you might need to define 

Signature: 
explain the input and output fields in an intuitive way with just few words.
("question"-> "answer")

Module:
Module can be something where you put the signatures into action by defining a certain module which compiles and generate response for you for the given query.

Construct a signaturre class, which defines the input fields and output fields needed. 
Also, give docstrings and description in verbose, so that the dspy signature could understand the context and compile best prompt for the usecase.

In [26]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts about ")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

Define the module with the actions needs to be performed, here we are showing a small RAG use case where we are retrieving similar contexts using our retriever class and generating response based on the factual context using one of the DSPy module `ChainOfThought`.

In [27]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

Now we are passing our query and retrieving relevant chunks using clarifai retriever and based on factual evidence, model is able to generate response.

In [28]:
# Ask any question you like to this RAG program.
my_question = "Which bid will be termed as L1?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
obj= RAG()
pred=obj(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: Which bid will be termed as L1 ?
Predicted Answer: The bidder who quotes the lowest price among
Retrieved Contexts (truncated): ['If L1 bid is not a ‘Class-I local supplier’, 50 (fifty) percent of the order quantity\nshall be awarded to L1. Thereafter, the lowest bidder among the ‘Class-I local\nsupplier’ will be invited to match ...', 'If L1 bid is not a ‘Class-I local supplier’, 50 (fifty) percent of the order quantity\nshall be awarded to L1. Thereafter, the lowest bidder among the ‘Class-I local\nsupplier’ will be invited to match ...', 'If L1 bid is not a ‘Class-I local supplier’, 50 (fifty) percent of the order quantity\nshall be awarded to L1. Thereafter, the lowest bidder among the ‘Class-I local\nsupplier’ will be invited to match ...']
